# Preparing the Dataset

In [1]:
import numpy as np
import cv2
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

rdhaar='/home/charles/opencv/data/haarcascades/'
rdmod = '/media/charles/1TB/Users/Charles Olivier/Downloads/ACHRI Models/vgg_face_caffe/'

from skimage import transform
model = rdmod+'VGG_FACE_deploy.prototxt'
weights = rdmod+'VGG_FACE.caffemodel'
# change this to your install dir
caffe_root = '~/Downloads/caffe-master/' 
import sys
sys.path.insert(0, caffe_root + 'python')
import caffe
mean = np.array([129.1863,104.7624,93.5940])

In [2]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])
def detect_face(img, thresh):
    faces = face_cascade.detectMultiScale(img, thresh, 1)
    try:
        f =faces.shape[0]==1
        return f, faces
    except AttributeError:
        return 0, ()

### Let's extract some faces

We will save the original image and the cropped faces in two different folders.

In [29]:
rdimg='./sad/'
tdir='./proc_sad/'
face_cascade = cv2.CascadeClassifier(rdhaar+'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(rdhaar+'haarcascade_eye.xml')
pos_count=0
tot_count=0
num=0
for file in os.listdir(rdimg):
    file_path=os.path.join(rdimg, file)
    tot_count+=1
    img = cv2.imread(file_path, 0)
    if img.shape:
        f=0
        thresh=1.4 
        while(f!=1 and thresh>1.0):
            f, faces = detect_face(img, thresh)
            thresh-=0.05

        if f==1:
            pos_count+=1
            for (x,y,w,h) in faces:
                img = cv2.imread(file_path, 1)
                dim=np.array([y-int(0.3*h),y+int(h*1.2), x-int(0.05*w),x+int(w*1.05)])
                dim[dim<0]=0
                crop_img = img[dim[0]:dim[1], dim[2]:dim[3],:]
                
                cv2.imwrite(tdir+'ori/'+str(num)+'.jpeg', img)
                cv2.imwrite(tdir+'crop/'+str(num)+'.jpeg', crop_img)
                num+=1
        else: continue

print('Percentage dataset converted: %d/%d = %.2f'%(pos_count, tot_count, pos_count/float(tot_count)))

Percentage dataset converted: 617/711 = 0.87


### Cleanup
Go through cropped faces manually and delete the failed ones. The script below will delete the corresponding originals in the second folder, as well as rename the files.

In [30]:
f=[]
for file in os.listdir(tdir+'crop'):
    f.append(file)

In [31]:
rm=0
for file in os.listdir(tdir+'ori'):
    if file not in f:
        os.remove(tdir+'ori/'+file)
        rm+=1
print('%d files removed', rm)

%d files removed 66


In [32]:
f=[]
for file in os.listdir(tdir+'crop'):
    f.append(file)

In [33]:
# rename files
name_count=0
for file in os.listdir(tdir+'crop'):
    os.rename(tdir+'crop/'+file, tdir+'crop/'+str(name_count)+'_.jpeg')
    os.rename(tdir+'ori/'+file, tdir+'ori/'+str(name_count)+'_.jpeg')
    name_count+=1

In [34]:
name_count=0
for file in os.listdir(tdir+'crop'):
    os.rename(tdir+'crop/'+file, tdir+'crop/'+str(name_count)+'.jpeg')
    os.rename(tdir+'ori/'+file, tdir+'ori/'+str(name_count)+'.jpeg')
    name_count+=1